<a href="https://colab.research.google.com/github/FNSY96/TerrorismEventsPrediction/blob/master/Graduation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Python Imports**
PLEASE PUT ALL LIBRARY IMPORTS IN THIS CELL **ONLY**

In [0]:
import pandas as pd
import csv
import glob
import os
from pandas import ExcelWriter
from pandas import ExcelFile
import io
print("imports run")

imports run


In [0]:
!pip3 install pandas

# Data Set Imports

In [0]:
!pip install -U -q PyDrive
from google.colab import drive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()

    100% |████████████████████████████████| 993kB 18.4MB/s 


In [0]:
# drive = GoogleDrive(gauth)
drive.mount("/content/drive", force_remount=True)
datadir  = 'drive/My Drive/GraduationProject/Data'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


### **GeoEPR**

In [0]:
# fid = drive.ListFile({'q':"title='GeoEPR-2018.1.csv'"}).GetList()[0]['id']
# f = drive.CreateFile({'id': fid})
# f.GetContentFile('GeoEPR-2018.1.csv')
# f.Upload()

### **Gecon40_post_final**

In [0]:
# fid = drive.ListFile({'q':"title='Gecon40_post_final_kh.csv'"}).GetList()[0]['id']
# f = drive.CreateFile({'id': fid})
# f.GetContentFile('Gecon40_post_final.csv')
# f.Upload()

###**GTD**

In [0]:
# !ls drive/'My Drive'/GraduationProject/Data/gtd.csv

# Pandas Data Frames
Useful link : https://youtu.be/2AFGPdNn4FM

## **GTD Data Frame**

In [0]:
gtd_df = pd.read_csv(os.path.join(datadir, 'gtd.csv'),encoding='latin-1')
print(gtd_df.columns.values)

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (4,6,31,33,61,62,63,76,79,90,92,94,96,114,115,121) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


['eventid' 'iyear' 'imonth' 'iday' 'approxdate' 'extended' 'resolution'
 'country' 'country_txt' 'region' 'region_txt' 'provstate' 'city'
 'latitude' 'longitude' 'specificity' 'vicinity' 'location' 'summary'
 'crit1' 'crit2' 'crit3' 'doubtterr' 'alternative' 'alternative_txt'
 'multiple' 'success' 'suicide' 'attacktype1' 'attacktype1_txt'
 'attacktype2' 'attacktype2_txt' 'attacktype3' 'attacktype3_txt'
 'targtype1' 'targtype1_txt' 'targsubtype1' 'targsubtype1_txt' 'corp1'
 'target1' 'natlty1' 'natlty1_txt' 'targtype2' 'targtype2_txt'
 'targsubtype2' 'targsubtype2_txt' 'corp2' 'target2' 'natlty2'
 'natlty2_txt' 'targtype3' 'targtype3_txt' 'targsubtype3'
 'targsubtype3_txt' 'corp3' 'target3' 'natlty3' 'natlty3_txt' 'gname'
 'gsubname' 'gname2' 'gsubname2' 'gname3' 'gsubname3' 'motive'
 'guncertain1' 'guncertain2' 'guncertain3' 'individual' 'nperps'
 'nperpcap' 'claimed' 'claimmode' 'claimmode_txt' 'claim2' 'claimmode2'
 'claimmode2_txt' 'claim3' 'claimmode3' 'claimmode3_txt' 'compclaim'


In [0]:
gtd_filtered_df = gtd_df[['country','city','latitude', 'longitude', 'target1', 'iyear', 'region']]
print(gtd_filtered_df)

        country                    city   latitude   longitude  \
0            58           Santo Domingo  18.456792  -69.951164   
1           130             Mexico city  19.371887  -99.086624   
2           160                 Unknown  15.478598  120.599741   
3            78                  Athens  37.997490   23.762728   
4           101                 Fukouka  33.580412  130.396361   
5           217                   Cairo  37.005105  -89.176269   
6           218              Montevideo -34.891151  -56.187214   
7           217                 Oakland  37.791927 -122.225906   
8           217                 Madison  43.076592  -89.412488   
9           217                 Madison  43.072950  -89.386694   
10          217                 Baraboo  43.468500  -89.744299   
11          217                  Denver  39.758968 -104.876305   
12           98                    Rome  41.890961   12.490069   
13          217                 Detroit  42.331685  -83.047924   
14        

In [0]:
# clean NaN
gtd_filtered_df = gtd_filtered_df.fillna(gtd_filtered_df.mean())

mode =  gtd_filtered_df['country'].mode()[0]
gtd_filtered_df['country'] = gtd_filtered_df.groupby('country')['country'].apply(lambda x: x.fillna(x.value_counts().idxmax() if x.value_counts().max() >=1 else mode , inplace = False))
gtd_filtered_df['country']= gtd_filtered_df['country'].fillna(gtd_filtered_df['country'].value_counts().idxmax())

mode =  gtd_filtered_df['city'].mode()[0]
gtd_filtered_df['city'] = gtd_filtered_df.groupby('city')['city'].apply(lambda x: x.fillna(x.value_counts().idxmax() if x.value_counts().max() >=1 else mode , inplace = False))
gtd_filtered_df['city']= gtd_filtered_df['city'].fillna(gtd_filtered_df['city'].value_counts().idxmax())

mode =  gtd_filtered_df['target1'].mode()[0]
gtd_filtered_df['target1'] = gtd_filtered_df.groupby('target1')['target1'].apply(lambda x: x.fillna(x.value_counts().idxmax() if x.value_counts().max() >=1 else mode , inplace = False))
gtd_filtered_df['target1']= gtd_filtered_df['target1'].fillna(gtd_filtered_df['target1'].value_counts().idxmax())

print(gtd_filtered_df)

        country                    city  \
0            58           Santo Domingo   
1           130             Mexico city   
2           160                 Unknown   
3            78                  Athens   
4           101                 Fukouka   
5           217                   Cairo   
6           218              Montevideo   
7           217                 Oakland   
8           217                 Madison   
9           217                 Madison   
10          217                 Baraboo   
11          217                  Denver   
12           98                    Rome   
13          217                 Detroit   
14          217             Rio Piedras   
15          499                  Berlin   
16           65                 Unknown   
17          217           New York City   
18          217              Rio Grande   
19          217                 Seattle   
20          217               Champaign   
21          218              Montevideo   
22         

## **GECON Data Frame**

In [0]:
gecon_df = pd.read_csv(os.path.join(datadir, 'Gecon40_post_final_kh.csv'),encoding='latin-1')
print(gecon_df.columns.values)

['AREA' 'COUNTRY' 'D1' 'D2' 'D3' 'D4' 'DIS_LAKE' 'DIS_MAJOR_RIVER'
 'DIS_OCEAN' 'DIS_RIVER' 'ELEV_SRTM' 'ELEV_SRTM_PRED' 'LAT' 'LONGITUDE'
 'LONG_NAME' 'MATTVEG' 'MER1990_40' 'MER1995_40' 'MER2000_40' 'MER2005_40'
 'NEWCOUNTRYID' 'POPGPW_1990_40' 'POPGPW_1995_40' 'POPGPW_2000_40'
 'POPGPW_2005_40' 'PPP1990_40' 'PPP1995_40' 'PPP2000_40' 'PPP2005_40'
 'PREC_NEW' 'PRECMAX' 'PRECMIN' 'PRECSD' 'PRECAVNEW80_08' 'PRECSDNEW80_08'
 'QUALITY' 'RIG_xi0710' 'ROUGH' 'SOIL_UNIT' 'TEMP_NEW' 'TEMPAV_8008'
 'TEMPMAX' 'TEMPMIN' 'TEMPSD' 'TEMPSD80_08' 'QUALITY_REVISION'
 'DATE OF LAST' 'Unnamed: 47' 'Unnamed: 48' 'Unnamed: 49' 'Unnamed: 50'
 'Unnamed: 51' 'Unnamed: 52' 'Unnamed: 53' 'Unnamed: 54' 'Unnamed: 55']


/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (16,19,25,28) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [0]:
gecon_filtered_df = gecon_df[['AREA', 'COUNTRY', 'DIS_LAKE','DIS_MAJOR_RIVER']]
print(gecon_filtered_df)

           AREA             COUNTRY  DIS_LAKE  DIS_MAJOR_RIVER
0             0  Bosnia&Herzegovina       NaN              NaN
1      10.47769              Brazil  156602.0         181754.2
2      33.62301              Brazil  580072.4         109199.3
3      12.15342              Brazil  630609.5         683543.5
4      36.46025              Brazil  615035.5         671245.9
5             0              Brazil  522919.3        1508678.0
6             0              Brazil  963283.0         441482.6
7      12.35509              Brazil  341525.0         461270.0
8      12.34964              Brazil  486620.9         511721.4
9             0            Cameroon  446003.6        1007535.0
10     361.9061            Cameroon    4701.4        2082788.0
11     7773.164              Canada  116347.6         731828.7
12     7773.164              Canada  153765.4         781171.1
13     7773.164              Canada  205771.2         833294.2
14     7773.164              Canada  263858.0         8

In [0]:
# clean NaN
gecon_filtered_df = gecon_filtered_df.fillna(gecon_filtered_df.mean())

print(gecon_filtered_df)

           DIS_LAKE  DIS_MAJOR_RIVER
0      1.574901e+06     2.868236e+06
1      1.566020e+05     1.817542e+05
2      5.800724e+05     1.091993e+05
3      6.306095e+05     6.835435e+05
4      6.150355e+05     6.712459e+05
5      5.229193e+05     1.508678e+06
6      9.632830e+05     4.414826e+05
7      3.415250e+05     4.612700e+05
8      4.866209e+05     5.117214e+05
9      4.460036e+05     1.007535e+06
10     4.701400e+03     2.082788e+06
11     1.163476e+05     7.318287e+05
12     1.537654e+05     7.811711e+05
13     2.057712e+05     8.332942e+05
14     2.638580e+05     8.877025e+05
15     2.097864e+05     6.459352e+05
16     1.453123e+05     6.591201e+05
17     2.158604e+05     7.561578e+05
18     1.549459e+05     7.672402e+05
19     2.657430e+05     8.666708e+05
20     2.200254e+05     8.761719e+05
21     3.454655e+05     9.773821e+05
22     3.123668e+05     9.856523e+05
23     2.818414e+05     1.035938e+06
24     2.886441e+05     1.060515e+06
25     6.950512e+05     1.114719e+06
2

## **GeoEPR Data Frame**

In [0]:
geo_df = pd.read_csv(os.path.join(datadir, 'GeoEPR-2018.1.csv'),encoding='latin-1')
# print(geo_df.columns.values)
# # statename, group
# print(geo_df)

In [0]:
geo_filtered_df = geo_df[['statename','group','type']]
print(geo_filtered_df)

                statename                    group              type
0           United States                   Whites         Statewide
1           United States        African Americans         Statewide
2           United States         American Indians  Regionally based
3           United States                   Whites         Statewide
4           United States         American Indians  Regionally based
5           United States        African Americans         Statewide
6           United States                   Whites         Statewide
7           United States         American Indians  Regionally based
8           United States        African Americans         Statewide
9           United States          Asian Americans             Urban
10          United States           Arab Americans             Urban
11          United States                  Latinos  Regional & urban
12                 Canada       Aboriginal peoples  Regionally based
13                 Canada         

In [0]:
# clean NaN
geo_filtered_df = geo_filtered_df.fillna(geo_filtered_df.mean())

mode =  geo_filtered_df['statename'].mode()[0]
geo_filtered_df['statename'] = geo_filtered_df.groupby('statename')['statename'].apply(lambda x: x.fillna(x.value_counts().idxmax() if x.value_counts().max() >=1 else mode , inplace = False))
geo_filtered_df['statename']= geo_filtered_df['statename'].fillna(geo_filtered_df['statename'].value_counts().idxmax())

mode =  geo_filtered_df['group'].mode()[0]
geo_filtered_df['group'] = geo_filtered_df.groupby('group')['group'].apply(lambda x: x.fillna(x.value_counts().idxmax() if x.value_counts().max() >=1 else mode , inplace = False))
geo_filtered_df['group']= geo_filtered_df['group'].fillna(geo_filtered_df['group'].value_counts().idxmax())

mode =  geo_filtered_df['type'].mode()[0]
geo_filtered_df['type'] = geo_filtered_df.groupby('type')['type'].apply(lambda x: x.fillna(x.value_counts().idxmax() if x.value_counts().max() >=1 else mode , inplace = False))
geo_filtered_df['type']= geo_filtered_df['type'].fillna(geo_filtered_df['type'].value_counts().idxmax())

print(geo_filtered_df)

                statename                    group              type
0           United States                   Whites         Statewide
1           United States        African Americans         Statewide
2           United States         American Indians  Regionally based
3           United States                   Whites         Statewide
4           United States         American Indians  Regionally based
5           United States        African Americans         Statewide
6           United States                   Whites         Statewide
7           United States         American Indians  Regionally based
8           United States        African Americans         Statewide
9           United States          Asian Americans             Urban
10          United States           Arab Americans             Urban
11          United States                  Latinos  Regional & urban
12                 Canada       Aboriginal peoples  Regionally based
13                 Canada         

## **Happiness World Report Data Frame**

In [0]:
happy_df = pd.read_csv(os.path.join(datadir, 'Happiness2017.csv'),encoding='latin-1')

In [0]:
happy_filtered_df = happy_df[['Country','Happiness.Rank','Happiness.Score']]
print(happy_filtered_df)

                      Country  Happiness.Rank  Happiness.Score
0                      Norway               1            7.537
1                     Denmark               2            7.522
2                     Iceland               3            7.504
3                 Switzerland               4            7.494
4                     Finland               5            7.469
5                 Netherlands               6            7.377
6                      Canada               7            7.316
7                 New Zealand               8            7.314
8                      Sweden               9            7.284
9                   Australia              10            7.284
10                     Israel              11            7.213
11                 Costa Rica              12            7.079
12                    Austria              13            7.006
13              United States              14            6.993
14                    Ireland              15          

In [0]:
# clean NaN
happy_filtered_df = happy_filtered_df.fillna(happy_filtered_df.mean())

mode =  happy_filtered_df['Country'].mode()[0]
happy_filtered_df['Country'] = happy_filtered_df.groupby('Country')['Country'].apply(lambda x: x.fillna(x.value_counts().idxmax() if x.value_counts().max() >=1 else mode , inplace = False))
happy_filtered_df['Country']= happy_filtered_df['Country'].fillna(happy_filtered_df['Country'].value_counts().idxmax())

print(happy_filtered_df)

                      Country  Happiness.Rank  Happiness.Score
0                      Norway               1            7.537
1                     Denmark               2            7.522
2                     Iceland               3            7.504
3                 Switzerland               4            7.494
4                     Finland               5            7.469
5                 Netherlands               6            7.377
6                      Canada               7            7.316
7                 New Zealand               8            7.314
8                      Sweden               9            7.284
9                   Australia              10            7.284
10                     Israel              11            7.213
11                 Costa Rica              12            7.079
12                    Austria              13            7.006
13              United States              14            6.993
14                    Ireland              15          

In [0]:
# print(happy_df.columns.values)
# print(happy_df['Country'])
# print(gtd_df.shape)

In [0]:
# import sys
# import math

# count1 = 0
# count2 = 0
# count3 = 0
# print(gtd_filtered_df.isna().sum())

# if happy_filtered_df["Country"] == "NaN":
#   print("fady")
# print(sys.getsizeof(happy_filtered_df["Country"][3]))

# for i in range(sys.getsizeof(happy_filtered_df["Country"])):
#   if happy_filtered_df["Country"][i] == "NaN":
#     count1 += 1
    
# for i in range(sys.getsizeof(happy_filtered_df["Happiness.Rank"])):
#   if math.isnan(happy_filtered_df["Happiness.Rank"][i]) :
#     count2 += 1
    
# for i in range(sys.getsizeof(happy_filtered_df["Happiness.Score"])):
#   if math.isnan(happy_filtered_df["Happiness.Score"][i]):
#     count3 += 1
  
# print(count1)
# print(count2)
# print(count3)


In [0]:
d = {'one': pd.Series([1., 3.], index=['a', 'c']), 'two': pd.Series([1., 2., 3., 4.], index=['a', 'b', 'c', 'd']), 'three': pd.Series(["A", "B", "A"], index=['a', 'c', 'd'])}
df = pd.DataFrame(d)
print(df)
# print(df.isna().sum())
df = df.fillna(df.mean())
mode =  df['three'].mode()[0]
df['three'] = df.groupby('three')['three'].apply(lambda x: x.fillna(x.value_counts().idxmax() if x.value_counts().max() >=1 else mode , inplace = False))
df['three']= df['three'].fillna(df['three'].value_counts().idxmax())
print(df)

   one three  two
a  1.0     A  1.0
b  NaN   NaN  2.0
c  3.0     B  3.0
d  NaN     A  4.0
   one three  two
a  1.0     A  1.0
b  2.0     A  2.0
c  3.0     B  3.0
d  2.0     A  4.0
